In [2]:
import tabula
import pdfminer
import camelot
import PyPDF2
import pandas as pd
import numpy as np
import re
from sorted_months_weekdays import Month_Sorted_Month
import streamlit as st
from datetime import datetime

In [3]:

def get_data_hdfc(path, password=None):
    with open(path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfFileReader(pdf_file)
        number = pdf_reader.getNumPages()
        meta_data = pdf_reader.documentInfo

    tables = camelot.read_pdf(path,
                              password=password,
                              flavor='stream',
                              pages='1-end',
                              columns=['50,275,365,420,491,570'],
                              edge_tol=-50,
                              encoding='utf-8'
                              # process_background=True
                              )

    dataframes = tables._tables

    for i in range(0, number):
        start = dataframes[i].df[dataframes[i].df.iloc[:, 0].str.contains('From :', case=True, regex=True)].index[0]
        end = \
        dataframes[i].df[dataframes[i].df.iloc[:, 0].str.contains('HDFC BANK LIMITED', case=True, regex=True)].index[0]
        dataframes[i].df = dataframes[i].df.iloc[start + 1:end]

    transaction_det = pd.DataFrame()

    for i in range(0, len(dataframes)):
        correct_transaction = pd.DataFrame(np.vstack([dataframes[i].df.columns, dataframes[i].df]))
        transaction_det = transaction_det.append(correct_transaction)
    transaction_det.drop(0, axis=0, inplace=True)
    transaction_det.columns = list(transaction_det.iloc[0])

    list(transaction_det.columns)

    transaction_det = transaction_det.iloc[1:]
    transaction_det.reset_index(drop=True, inplace=True)

    trial = transaction_det.copy()
    a = range(len(trial))
    start = a.start
    stop = trial[trial.iloc[:, 1].str.contains('STATEMENT SUMMARY|TRANSACTION TOTAL', case=True, regex=True)].index[0]

    for i in range(stop - 1, start - 1, -1):
        while trial.iloc[i, -1] == '':
            trial.iloc[i - 1, 1] = str(trial.iloc[i - 1, 1]) + str(trial.iloc[i, 1])
            trial.iloc[i - 1, 2] = str(trial.iloc[i - 1, 2]) + str(trial.iloc[i, 2])
            trial.drop(trial.index[i], axis=0, inplace=True)
            break
    trial.reset_index(drop=True, inplace=True)

    def replace_comma(text):
        text = re.sub(',', '', str(text))
        return text

    names = list(trial.columns)

    numeric_dtypes_index = []
    date_dtypes_index = []
    for x in names:
        if re.search('Debit|Credit|Balance|Withdrawal|Deposit', x):
            numeric_dtypes_index.append(names.index(x))
        elif re.search('Date|DATE|Value Dt', x):
            date_dtypes_index.append(names.index(x))

    for i in numeric_dtypes_index:
        trial.iloc[:, i] = trial.iloc[:, i].apply(replace_comma)
        trial.iloc[:, i] = pd.to_numeric(trial.iloc[:, i], errors='coerce')

    stop = trial[trial.iloc[:, 1].str.contains('STATEMENT SUMMARY|TRANSACTION TOTAL', case=True, regex=True)].index[0]
    for i in date_dtypes_index:
        trial.iloc[:stop, i] = pd.to_datetime(trial.iloc[:stop, i], format='%d/%m/%y').dt.date

    for i in range(len(names)):
        if re.search('Narration|Remarks|Description|Details|Particulars', str(names[i])):
            names[i] = 'Transaction Details'

    trial.columns = names

    return trial


In [4]:

def get_data_sbi(path, password=None):
    rows = tabula.read_pdf(path,
                           encoding="utf-8",
                           pages="all",
                           stream=True,
                           # area = (578.57,23.6,768.36, 546.96),
                           guess=False,
                           lattice=True,
                           password=password
                           )

    transaction_det = pd.DataFrame()

    
    for i in range(len(rows)):
        try:
            correct_transaction = pd.DataFrame(np.vstack([rows[i].columns, rows[i]]))
            correct_transaction.dropna(subset=[0, 1], inplace=True)
            transaction_det = transaction_det.append(correct_transaction)
        except:
            next


    def replace_comma(text):
        text = re.sub(',', '', str(text))
        return text

    def replace_hyphen(text):
        text = re.sub('\-|nan', '0', str(text))
        return text

    transaction_det.replace('\r', ' ', regex=True, inplace=True)
    transaction_det = transaction_det.rename(columns=transaction_det.iloc[0]).drop(
        transaction_det.index[0]).reset_index(
        drop=True)
    names = list(transaction_det.columns)

    numeric_dtypes_index = []
    numeric_dtypes_cols = []
    for x in names:
        if re.findall('Debit|Credit|Balance|Withdrawal|Deposit', str(x)):
            numeric_dtypes_index.append(names.index(x))
            numeric_dtypes_cols.append(x)
    list(transaction_det)

    # transaction_det
    for i in numeric_dtypes_index:
        transaction_det.iloc[:, i] = transaction_det.iloc[:, i].apply(replace_hyphen)
        transaction_det.iloc[:, i] = transaction_det.iloc[:, i].apply(replace_comma)
        transaction_det.iloc[:, i] = pd.to_numeric(transaction_det.iloc[:, i], errors='coerce', downcast='float')

    date_dtypes_index = []
    for x in names:
        if re.findall('Date|DATE', str(x)):
            date_dtypes_index.append(names.index(x))

    for i in date_dtypes_index:
        transaction_det.iloc[:, i] = pd.to_datetime(transaction_det.iloc[:, i], format='%d %b %Y',
                                                    errors='ignore').dt.date

    list(transaction_det)
    # transaction_det['Ref No./Cheque No'] = transaction_det['Ref No./Cheque No'].astype('string')

    cheque_index = []
    for x in names:
        if re.findall('Cheque|Ref', str(x)):
            cheque_index.append(names.index(x))
    i = cheque_index[0]

    # list(transaction_det)
    # transaction_det['Ref No./Cheque No.'][transaction_det.iloc[:, 2].notna()].dtype

    # for i in cheque_index:
    if type(transaction_det.iloc[0, i]) != str:
        transaction_det.iloc[:, i] = pd.to_numeric(transaction_det.iloc[:, i], errors='coerce', downcast='integer')
        transaction_det.iloc[:, i].fillna(0, inplace=True)
        transaction_det.iloc[:, i] = transaction_det.iloc[:, i].astype(int)
        transaction_det.iloc[:, i] = transaction_det.iloc[:, i].astype(str)

    names = list(transaction_det.columns)
    for i in range(len(names)):
        if re.search('Description|Details|Particulars', str(names[i])):
            names[i] = 'Transaction Details'
    transaction_det.columns = names
    transaction_det[numeric_dtypes_cols] = transaction_det[numeric_dtypes_cols].replace({0: np.nan})
    return transaction_det


In [5]:

def get_data_icici(path , password=None):
    rows = tabula.read_pdf(path ,
                           pages="all",
                           lattice=True,
                           stream=True,
                           password=password,
                           #pandas_options={'header': None}
                           )
    transaction_det = pd.DataFrame()
    for i in range(len(rows)):
        try:
            correct_transaction = pd.DataFrame(np.vstack([rows[i].columns, rows[i]]))
            # correct_transaction.dropna(subset = [0,1],inplace=True)
            transaction_det = transaction_det.append(correct_transaction)
        except:
            next

    transaction_det.drop(0, axis=1, inplace=True)
    transaction_det.dropna(how='all', inplace=True)
    if transaction_det.iloc[0,0] == "Value Date":
        transaction_det.columns = list(transaction_det.iloc[0])
        transaction_det = transaction_det.iloc[1:]
        transaction_det.reset_index(drop=True, inplace=True)
    elif transaction_det.iloc[1,0] == "Value Date":
        transaction_det.columns = list(transaction_det.iloc[1])
        transaction_det = transaction_det.iloc[2:]
        transaction_det.reset_index(drop=True, inplace=True)

    names = list(transaction_det.columns)
    numeric_dtypes_index = []
    numeric_dtypes_cols = []
    for x in names:
        if re.findall('Debit|Credit|Balance|Withdrawal|Deposit|Init', str(x)):
            numeric_dtypes_index.append(names.index(x))
            numeric_dtypes_cols.append(x)

    for i in numeric_dtypes_index:
        transaction_det.iloc[:, i] = pd.to_numeric(transaction_det.iloc[:, i], errors='coerce')

    for i in range(len(names)):
        if re.search('Remarks|Description|Details|Particulars', str(names[i])):
            names[i] = 'Transaction Details'
            value = i
    transaction_det.columns = names

    date_dtypes_index = []
    for x in names:
        if re.findall('Date|DATE', str(x)):
            date_dtypes_index.append(names.index(x))

    trial = transaction_det.copy()
    a = range(len(trial))
    start = a.start
    stop = a.stop

    for i in range(stop - 1, start - 1, -1):
        while np.isnan(trial.iloc[i, -1]) == True:
            trial.iloc[i - 1, value] = str(trial.iloc[i - 1, value]) + '' + str(trial.iloc[i, value])
            trial.drop(trial.index[i], axis=0, inplace=True)
            break
    trial.reset_index(drop=True, inplace=True)

    for i in range(len(trial)):
        for j in date_dtypes_index:
            date = re.search('[\d]{1,2}/[\d]{1,2}/[\d]{4}', str(trial.iloc[i, j]))
            trial.iloc[i, j] = date[0]

    for i in date_dtypes_index:
        trial.iloc[:, i] = pd.to_datetime(trial.iloc[:, i], format='%d/%m/%Y', errors='ignore').dt.date

    trial[numeric_dtypes_cols] = trial[numeric_dtypes_cols].replace({0: np.nan})
    return trial

In [ ]:
def get_data_pnb(path, password=None):
    rows = tabula.read_pdf(path,
                           pages="all",
                           lattice=True,
                           password=None
                           )

    transaction_det = pd.DataFrame()
    transaction_det = transaction_det.append(rows)
    transaction_det.reset_index(drop=True, inplace=True)
    transaction_det.replace('\r', ' ', regex=True, inplace=True)

    transaction_det.rename(columns={'Transaction\rDate': 'Value Date', 'Cheque\rNumber': 'Cheque',
                                    'Narration': 'Transaction Details'}, inplace=True)

    transaction_det['Value Date'] = pd.to_datetime(transaction_det['Value Date'], errors='ignore', format = "%d/%m/%Y")
    transaction_det['Value Date'] = transaction_det['Value Date'].dt.date
#    transaction_det['Value Date'] = transaction_det['Value Date'].strftime("%d/%m/%Y")

    transaction_det['Cheque'] = transaction_det['Cheque'].replace(np.nan, 0)
    transaction_det['Cheque'] = transaction_det['Cheque'].astype(int)

    transaction_det['Balance'] = transaction_det['Balance'].map(lambda x: x.rstrip('Cr.'))
    transaction_det['Balance'] = transaction_det['Balance'].astype(float).round(4)

    transaction_det['Transaction Details'] = transaction_det['Transaction Details'].map(lambda x: x.rstrip('\\'))
    transaction_det['Transaction Details'] = transaction_det['Transaction Details'].str.replace('+', '')

    transaction_det = transaction_det.reindex(
        columns=['Value Date', 'Transaction Details', 'Cheque', 'Deposit', 'Withdrawal', 'Balance'])
    trial = transaction_det.copy()
    return trial


In [6]:

def get_data_cbi(path , password = None) :

    rows = tabula.read_pdf(path,
                               pages="all",
                               lattice=True,
                               password=None
                               )

    transaction_det = pd.DataFrame()
    transaction_det = transaction_det.append(rows)
    transaction_det.reset_index(drop=True, inplace=True)
    transaction_det.replace('\r', ' ', regex=True, inplace=True)

#    df_columns = list(transaction_det.columns)

    transaction_det = transaction_det.rename(columns= {'Cheque\rNumber' : 'Cheque' , 'Value\rDate' : 'Value Date',
                                                       'Debit' : 'Withdrawal', 'Credit' : 'Deposit',
                                                       'Account Description' : 'Transaction Details'})

    transaction_det = transaction_det.drop(['Post Date', 'Branch\rCode'], axis = 1)
    transaction_det = transaction_det[['Value Date', 'Transaction Details', 'Cheque', 'Deposit', 'Withdrawal', 'Balance']]

    transaction_det['Value Date'] = pd.to_datetime(transaction_det['Value Date'], errors= 'ignore')
    transaction_det['Value Date'] = transaction_det['Value Date'].dt.date

    transaction_det['Cheque'] = transaction_det['Cheque'].replace(np.nan, 0)
    transaction_det['Cheque'] = transaction_det['Cheque'].astype(int)

    transaction_det['Balance'] = transaction_det['Balance'].map(lambda x: x.rstrip('CR'))
    transaction_det['Balance'] = transaction_det['Balance'].astype(float).round(4)

    trial = transaction_det.copy()
    return trial


In [ ]:

def get_data_idbi(path, password=None):
    rows = tabula.read_pdf(path,
                           pages="all",
                           lattice=False,
                           password=None
                           )

    transaction_det = pd.DataFrame()
    transaction_det = transaction_det.append(rows)
    df_columns = transaction_det.columns

    transaction_det = transaction_det.drop(index=0)
    transaction_det = transaction_det.drop(
        columns={'Srl', 'Txn Date', 'CCY', 'Dr Count', 'Cr Count', 'Debits', 'Credits'})

    transaction_det['Withdrawal'] = ''
    transaction_det['Deposit'] = ''

    transaction_det['Cheque'] = transaction_det['Cheque'].replace(np.nan, 0)
    transaction_det = transaction_det.dropna(subset=['Value Date'])
    transaction_det.reset_index(drop=True, inplace=True)

    transaction_det['Value Date'] = pd.to_datetime(transaction_det['Value Date'], errors='ignore', format = "%d/%m/%y")
    transaction_det['Value Date'] = transaction_det['Value Date'].dt.date

    transaction_det['Deposit'] = transaction_det.loc[transaction_det['CR/DR'] == 'CR', 'Trxn Amount']
    transaction_det['Deposit'] = transaction_det['Deposit'].str.replace(",", "").astype(float).round(4)

    transaction_det['Withdrawal'] = transaction_det.loc[transaction_det['CR/DR'] == 'DR', 'Trxn Amount']
    transaction_det['Withdrawal'] = transaction_det['Withdrawal'].str.replace(",", "").astype(float).round(4)

    transaction_det['Balance'] = transaction_det['Balance'].str.replace(",", "").astype(float).round(4)

    transaction_det = transaction_det.drop(columns={'CR/DR', 'Trxn Amount'})
    transaction_det.rename(columns={'Description': 'Transaction Details'}, inplace=True)

    transaction_det = transaction_det.reindex(
        columns=['Value Date', 'Transaction Details', 'Cheque', 'Deposit', 'Withdrawal', 'Balance'])
    trial = transaction_det.copy()
    return trial


In [ ]:

def get_data_standardchartered(path , password=None):
    rows = tabula.read_pdf(path,
                           pages="all",
                           # stream=True,
                           # # area = (578.57,23.6,768.36, 546.96),
                           # guess=True,
                           lattice=False,
                           password=password
                           )

    transaction_det = pd.DataFrame()
    for i in range(len(rows)):
        for j in range(len(rows[i])):
            try:
                if re.search('Cheque', str(rows[i].iloc[j, 1])):
                    correct_transaction = pd.DataFrame(np.vstack([rows[i].columns, rows[i].iloc[j:]]))
                    transaction_det = transaction_det.append(correct_transaction)
            except:
                next

    transaction_det.replace('\r', ' ', regex=True, inplace=True)
    transaction_det.drop(transaction_det.index[0], inplace=True)
    transaction_det = transaction_det.rename(columns=transaction_det.iloc[0]).drop(
        transaction_det.index[0]).reset_index(drop=True)

    transaction_det.columns = ['Date   Value Description', 'Cheque', 'Deposit', 'nan', 'Withdrawal', 'Balance']
    transaction_det.drop('nan', axis=1, inplace=True)

    names = list(transaction_det.columns)

    def replace_comma(text):
        text = re.sub(',', '', str(text))
        return text

    numeric_dtypes_index = []
    for x in names:
        if re.findall('Debit|Credit|Balance|Withdrawal|Deposit', str(x)):
            numeric_dtypes_index.append(names.index(x))

    # transaction_det
    for i in numeric_dtypes_index:
        # transaction_det.iloc[:, i] = transaction_det.iloc[:, i].apply(replace_hyphen)
        transaction_det.iloc[:, i] = transaction_det.iloc[:, i].apply(replace_comma)
        transaction_det.iloc[:, i] = pd.to_numeric(transaction_det.iloc[:, i], errors='coerce')

    transaction_det['Date   Value Description'] = transaction_det['Date   Value Description'].astype(str)

    cheque_index = []
    for x in names:
        if re.findall('Cheque|Ref', str(x)):
            cheque_index.append(names.index(x))
    i = cheque_index[0]

    if type(transaction_det.iloc[0, i]) != str:
        transaction_det.iloc[:, i] = pd.to_numeric(transaction_det.iloc[:, i], errors='coerce', downcast='integer')
        transaction_det.iloc[:, i].fillna(0, inplace=True)
        transaction_det.iloc[:, i] = transaction_det.iloc[:, i].astype(int)
        transaction_det.iloc[:, i] = transaction_det.iloc[:, i].astype(str)

    trial = transaction_det.copy()

    # i=0
    a = range(len(trial))
    start = a.start
    stop = a.stop

    for i in range(stop - 1, start - 1, -1):
        if re.search('Date|Balance Brought Forward', str(trial.iloc[i, 0])):
            continue
        while np.isnan(trial.iloc[i, -1]) == True:
            trial.iloc[i - 1, 0] = str(trial.iloc[i - 1, 0]) + ' ' + str(trial.iloc[i, 0])
            trial.drop(trial.index[i], axis=0, inplace=True)
            break
    trial.reset_index(drop=True, inplace=True)

    trial.iloc[:, 0] = trial.iloc[:, 0].str.replace('^(Date)', 'nan')
    x = list(trial.iloc[:, 0][trial.iloc[:, 0].str.contains('nan')].index)
    for i in x:
        trial.drop(i, axis=0, inplace=True)
    trial.reset_index(drop=True, inplace=True)

    trial['Value Date'] = ''
    trial['Transaction Details'] = ''
    year = 2020
    for i in range(len(trial)):
        dates = []
        date = None
        for j in re.finditer('(?i)(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s[\d]{2}', str(trial.iloc[i, 0])):
            dates.append(j)
        if len(dates) > 1:
            date = dates[1].group()
            trial.loc[i, 'Value Date'] = str(date) + ' ' + str(year)
            trial.loc[i, 'Transaction Details'] = str(trial.iloc[i, 0]).split(str(date))[-1].strip()
        elif len(dates) == 1:
            date = dates[0].group()
            trial.loc[i, 'Value Date'] = str(date) + ' ' + str(year)
            trial.loc[i, 'Transaction Details'] = str(trial.iloc[i, 0]).split(str(date))[-1].strip()
        else:
            date = 'Not Found'
            trial.loc[i, 'Value Date'] = str(date)
            trial.loc[i, 'Transaction Details'] = str(trial.iloc[i, 0])

    trial.drop(list(trial.columns)[0], axis=1, inplace=True)
    trial = trial.reindex(columns=['Value Date', 'Transaction Details', 'Cheque', 'Deposit', 'Withdrawal', 'Balance'])

    for i in range(len(trial)):
        if re.search('Not Found', str(trial.iloc[i, 0])):
            continue
        else:
            trial.iloc[i, 0] = pd.to_datetime(trial.iloc[i, 0].strip(), format='%b %d %Y', errors='ignore').date()

    for i in range(len(trial)):
        if re.search('Not Found', str(trial.iloc[i, 0])):
            trial.iloc[i, 0] = trial.iloc[i + 1, 0]
        else:
            continue
    return trial


In [7]:

def get_data_IndianBank(path , password = None) :

    rows = tabula.read_pdf(path,
                               pages="all",
                               lattice= True,
                               stream = True,
                               password=None
                           )

    transaction_det = pd.DataFrame()

    for i in range(len(rows)):
        try:
            correct_transaction = pd.DataFrame(np.vstack([rows[i].columns, rows[i]]))
            transaction_det = transaction_det.append(correct_transaction)
        except:
            next

    transaction_det.columns = transaction_det.iloc[0]
    transaction_det = transaction_det.drop([0])

    transaction_det.reset_index(drop=True, inplace=True)


    transaction_det['Balance'] = transaction_det['Balance'].astype(str).str.replace('Cr', '')
    transaction_det['Balance'] = transaction_det['Balance'].astype(str).str.replace('Dr', '')
    transaction_det['Balance'] = transaction_det['Balance'].astype(float).round(4)

    transaction_det['Chq No.'] = transaction_det['Chq No.'].replace(np.nan, 0)

    transaction_det.rename(columns={'Value Dt.' : 'Value Date', 'Details' : 'Transaction Details',
                                    'Chq No.' : 'Cheque', 'Debit' : 'Withdrawal', 'Credit' : 'Deposit'}, inplace= True)

    transaction_det['Withdrawal'] = transaction_det['Withdrawal'].astype(float).round(4)
    transaction_det['Deposit'] = transaction_det['Deposit'].astype(float).round(4)

    transaction_det['Post Dt.'] = transaction_det['Post Dt.'].replace(np.nan, '')
    transaction_det['Value Date'] = transaction_det['Value Date'].replace(np.nan, '')
    transaction_det['Transaction Details'] = transaction_det['Transaction Details'].replace(np.nan, '')

    transaction_det['Post Dt.'] = transaction_det['Post Dt.'].str.replace('\d+', '')
    transaction_det['Post Dt.'] = transaction_det['Post Dt.'].str.replace('//', '')

    value_dates = transaction_det['Value Date']

    value_dates = value_dates.str.replace('[a-zA-Z]', '')
    value_dates = value_dates.str.replace('(', '')
    value_dates = value_dates.str.replace(':', '')
    value_dates.replace('', float('NaN'), inplace=True)
    value_dates.replace(' ', float('NaN'), inplace=True)
    value_dates.replace('  ', float('NaN'), inplace=True)
    value_dates.dropna(inplace = True)
    value_dates.reset_index(drop = True , inplace = True)

    for i in range(len(value_dates)):
        format_str_1 = '%d/%m/%Y'
        format_str_2 = '%d/%m/%y'
        try :
            value_dates[i] = datetime.strptime(value_dates[i], format_str_1)
        except ValueError :
            value_dates[i] = datetime.strptime(value_dates[i], format_str_2)

    value_dates = value_dates.dt.date

    transaction_det['Value_Date_1'] = transaction_det['Value Date']
    transaction_det['Value_Date_1'] = pd.to_datetime(transaction_det['Value Date'], errors = 'coerce').dt.date.astype(str)
    transaction_det.loc[transaction_det['Value Date'].str.contains('/') , 'Value Date'] = transaction_det.loc[transaction_det['Value Date'].str.contains('/'), 'Value_Date_1']

    transaction_det = transaction_det[~transaction_det['Transaction Details'].str.contains("eposit by SELF")]
    transaction_det['Transaction Details'] = transaction_det['Transaction Details'].replace('Y CASH DEPOSIT', 'Y CASH DEPOSIT  deposit by SELF')

    transaction_det_1 = transaction_det.copy()

    transaction_det_1['Value Date'] = transaction_det_1['Value Date'][~transaction_det_1['Value Date'].str.contains('BROUGHT FORWARD')]
    transaction_det_1['Value Date'] = transaction_det_1['Value Date'].replace('CLOSING BALANCE :', np.nan)
    transaction_det_1.dropna(subset=['Value Date'],inplace=True)


    txn_det = transaction_det_1.iloc[:, 0] + transaction_det_1.iloc[:, 1]  + \
                    transaction_det_1.iloc[:, 2]

    transaction_det_1['Value_Date_1'].replace('NaT', np.nan, inplace=True)
    transaction_det_1.dropna(subset=['Value_Date_1'],inplace=True)

    txn_det_list = txn_det.tolist()
    join_list = ''.join(txn_det_list)
    join_list_new = re.split('[\d]{4}-[\d]{1,2}-[\d]{1,2}', join_list)
    txn_det_new = pd.DataFrame(join_list_new)

    txn_det_new.replace('', np.nan, inplace=True)
    txn_det_new.dropna(axis=0, inplace= True)
    txn_det_new.reset_index(drop=True, inplace=True)

    transaction_det_1.drop(columns= ['Post Dt.', 'Value_Date_1', 'Transaction Details'], inplace= True)
    transaction_det_1.reset_index(drop=True, inplace=True)
    transaction_det_1['Transaction Details'] = txn_det_new

    transaction_det_1 = transaction_det_1.reindex(
        columns=['Value Date', 'Transaction Details', 'Cheque', 'Deposit', 'Withdrawal', 'Balance'])

    transaction_det_1['Cheque'] = transaction_det_1['Cheque'].astype(int)

    transaction_det_1.rename(columns={'Value Dt' : 'Value Date'}, inplace= True)

    transaction_det_1.drop(columns= ['Value Date'], inplace= True)
    transaction_det_1['Value Date'] = value_dates

    transaction_det_1 = transaction_det_1.reindex(
        columns=['Value Date', 'Transaction Details', 'Cheque', 'Deposit', 'Withdrawal', 'Balance'])

    trial = transaction_det_1.copy()
    return trial


In [ ]:

def preprocess_columns(trial):
    og_names = list(trial.columns)
    names = og_names

    for i in range(len(names)):
        if re.search('Narration|Remarks|Description|Details|Particulars', str(names[i])):
            names[i] = 'Transaction Details'
        elif re.search('Cheque|Chq', str(names[i])):
            names[i] = 'Chq./Ref.No.'
        elif re.search('Debit|Withdrawal', str(names[i])):
            names[i] = 'Withdrawal Amt.'
        elif re.search('Credit|Deposit', str(names[i])):
            names[i] = 'Deposit Amt.'
        elif re.search('Balance', str(names[i])):
            names[i] = 'Closing Balance'

    if re.search('Value Date', str(names)):
        for i in range(len(names)):
            if re.search('Value Date', str(names[i])):
                names[i] = 'Value Dt'
    elif re.search('Transaction Date', str(names)):
        for i in range(len(names)):
            if re.search('Transaction Date', str(names[i])):
                names[i] = 'Value Dt'
    else:
        for i in range(len(names)):
            if re.search('Date', str(names[i])):
                names[i] = 'Value Dt'

    trial.columns = names
    try:
        stop = trial[trial.loc[:, 'Transaction Details'].str.contains('STATEMENT SUMMARY|TRANSACTION TOTAL', case=True,
                                                                      regex=True)].index[0]
    except IndexError:
        stop = len(trial)

    return trial, stop


In [ ]:

def get_preprocessing_data(path, key, password=None):
    trial = None
    stop = None
    if key == 'AXIS':
        trial = get_data_axis(path, password)
        trial, stop = preprocess_columns(trial)
    elif key == 'ICICI':
        trial = get_data_icici(path)
        trial, stop = preprocess_columns(trial)
    elif key == 'SBI':
        trial = get_data_sbi(path)
        trial, stop = preprocess_columns(trial)
    elif key == 'STANDARD CHARTERED':
        trial = get_data_standardchartered(path)
        trial, stop = preprocess_columns(trial)
    elif key == 'HDFC':
        trial = get_data_hdfc(path)
        try:
            stop = trial[trial.loc[:, 'Transaction Details'].str.contains('STATEMENT SUMMARY|TRANSACTION TOTAL', case=True,
                                                                   regex=True)].index[0]
        except IndexError:
            stop = len(trial)
        trial = trial[0:stop]
    elif key == 'PNB':
        trial = get_data_pnb(path)
        trial, stop = preprocess_columns(trial)
    elif key == 'Central Bank Of India':
        trial = get_data_cbi(path)
        trial, stop = preprocess_columns(trial)
    elif key == 'IDBI':
        trial = get_data_idbi(path)
        trial, stop = preprocess_columns(trial)
    elif key == 'Indian Bank':
        trial = get_data_IndianBank(path)
        trial, stop = preprocess_columns(trial)
 
    return trial, stop

def alpha_num_string(x):
    alpha_num_list = []
    alpha_num_list = re.sub('[\W_]+', ' ', x).split()
    return alpha_num_list


In [8]:

def preprocess(trial, stop):
    bank_path = 'files/'
    for i in range(1, stop):
        j = trial.loc[i, 'Value Dt'].strftime('%d%m%y')
        if j == trial.loc[i - 1, 'Value Dt'].strftime('%d%m%y'):
            continue
        elif j > trial.loc[i - 1, 'Value Dt'].strftime('%d%m%y'):
            flag = 'Dont Sort'
            break
        elif j < trial.loc[i - 1, 'Value Dt'].strftime('%d%m%y'):
            flag = 'Sort'
            break

    if flag == 'Sort':
        x = trial.iloc[0:stop].sort_index(ascending=False, axis=0)
        x.reset_index(drop=True, inplace=True)
        trial.iloc[0:stop] = x
        trial.reset_index(drop=True, inplace=True)
    else:
        pass

    trial['Correct_Transaction'] = ''

    names = list(trial.columns)
    numeric_dtypes_index = []
    for x in names:
        if re.findall('Debit|Credit|Balance|Withdrawal|Deposit|Init', str(x)):
            numeric_dtypes_index.append(names.index(x))

    for i in numeric_dtypes_index:
        trial.iloc[0:stop, i].fillna(0, inplace=True)

    for i in range(1, stop):
        if int(trial.loc[i, 'Deposit Amt.']) == 0 and int(trial.loc[i, 'Closing Balance']) in \
                range(int(trial.loc[i - 1, 'Closing Balance'] - trial.loc[i, 'Withdrawal Amt.']) - 2,
                      int(trial.loc[i - 1, 'Closing Balance'] - trial.loc[i, 'Withdrawal Amt.']) + 2):
            trial.loc[i, 'Correct_Transaction'] = True
        elif int(trial.loc[i, 'Withdrawal Amt.']) == 0 and \
                int(trial.loc[i, 'Closing Balance']) in \
                range(int(trial.loc[i - 1, 'Closing Balance'] + trial.loc[i, 'Deposit Amt.']) - 2,
                      int(trial.loc[i - 1, 'Closing Balance'] + trial.loc[i, 'Deposit Amt.']) + 2):
            trial.loc[i, 'Correct_Transaction'] = True

    for i in numeric_dtypes_index:
        for j in range(0, stop):
            if int(trial.iloc[j, i]) == 0:
                trial.iloc[j, i] = np.nan

    trial['TransactionType'] = ''
    trial['Transaction Details'] = trial['Transaction Details'].str.upper()

    trial.loc[:, 'Tokens'] = trial.loc[:, 'Transaction Details'].apply(alpha_num_string)

    atm_cash_transactions_list = ['ATM', 'CASH', 'VAT', 'MAT', 'NFS']
    loan_emi_transactions_list = ['EMI', 'NACH', 'LNPY', 'ACH']
    e_commerce_transactions_list = ['ECOM', 'PUR', 'VSI', 'IIN', 'VIN']
    pos_transactions_list = ['POS', 'BRN', 'VPS', 'IPS']
    online_bft_transactions_list = ['NEFT', 'IMPS', 'RTGS', 'BIL', 'INF', 'INFT', 'TPT']
    upi_transactions_list = ['UPI']
    cheque_transactions_list = ['CLG', 'CMS']
    mob_transactions_list = ['MOB', 'MMT']
    tax_ded_transactions_list = ['SGST', 'CGST']
    salary_transactions_list = ['SALARY', 'BONUS', 'REIMBURSEMENT']
    penalty_transactions_list = ['MABCHGS']
    
    
    for i in trial.index:
        if pd.Series(trial.loc[i, 'Tokens']).isin(atm_cash_transactions_list).any():
            trial.loc[i, 'TransactionType'] = 'ATM_CASH_TRANSACTION'
        if pd.Series(trial.loc[i, 'Tokens']).isin(loan_emi_transactions_list).any():
            trial.loc[i, 'TransactionType'] = 'LOAN_EMI'
        elif pd.Series(trial.loc[i, 'Tokens']).isin(e_commerce_transactions_list).any():
            trial.loc[i, 'TransactionType'] = 'E-COMMERCE_TRANSACTION'
        elif pd.Series(trial.loc[i, 'Tokens']).isin(pos_transactions_list).any():
            trial.loc[i, 'TransactionType'] = 'POINTOFSALE_DCT'
        elif pd.Series(trial.loc[i, 'Tokens']).isin(online_bft_transactions_list).any():
            trial.loc[i, 'TransactionType'] = 'ONLINE_BANK_FUNDTRANSFER'
        elif pd.Series(trial.loc[i, 'Tokens']).isin(upi_transactions_list).any():
            trial.loc[i, 'TransactionType'] = 'UPI'
        elif pd.Series(trial.loc[i, 'Tokens']).isin(cheque_transactions_list).any():
            trial.loc[i, 'TransactionType'] = 'CHEQUE_TRANSACTION'
        elif re.search('INT.PD|INT.COLL|INTEREST', trial.loc[i, 'Transaction Details']):
            trial.loc[i, 'TransactionType'] = 'INTEREST_COLLECTION'
        elif pd.Series(trial.loc[i, 'Tokens']).isin(mob_transactions_list).any():
            trial.loc[i, 'TransactionType'] = 'MOBILE_TRANSFER'
        elif re.search('OPENING BALANCE|BALANCE BROUGHT FORWARD', trial.loc[i, 'Transaction Details']):
            trial.loc[i, 'TransactionType'] = 'OPENING BALANCE'
        elif pd.Series(trial.loc[i, 'Tokens']).isin(tax_ded_transactions_list).any():
            trial.loc[i, 'TransactionType'] = 'TAX_DEDUCTION'
        if pd.Series(trial.loc[i, 'Tokens']).isin(salary_transactions_list).any():
            trial.loc[i, 'TransactionType'] = 'SALARY_BONUS'
        elif pd.Series(trial.loc[i, 'Tokens']).isin(penalty_transactions_list).any():
            trial.loc[i, 'TransactionType'] = 'PENALTY_CHARGES'

    trial.drop('Tokens', axis=1, inplace=True)

    bankslist = pd.read_csv(bank_path + 'Bankslist.csv')
    bankcodes = list(bankslist['Bank_Code'])
    banknames = list(bankslist['Bank_Name'])
    bank_code_name = dict(zip(bankcodes, banknames))
    added_providers = {'HEROFINC': 'Hero FinCorp.', 'RAZORPAY': 'Razor Pay', 'BAJAJFIN': 'Bajaj Financial',
                       'AXIS': 'Axis Bank'}

    bank_code_name.update(added_providers)
    trans_val = list(trial.columns).index('Transaction Details')

    trial = trial.sort_values(["Value Dt"])
    
    trial['Month'] = ''
    for i in range(0, stop):
        trial.loc[i, 'Month'] = trial.loc[i, 'Value Dt'].strftime('%B')
        trial.loc[i, 'Year'] = trial.loc[i, 'Value Dt'].strftime('%Y')
        if pd.Series(trial.loc[i,"Value Dt"].strftime('%d')).isin(["01","08","15","25"]).any():
            trial.loc[i,"AvgWithDrawal"] = trial.loc[i,"Withdrawal Amt."]
        else:
            trial.loc[i,"AvgWithDrawal"] = 0
        
        if pd.Series(trial.loc[i,"Value Dt"].strftime('%d')).isin(["01","08","15","25"]).any():
            trial.loc[i,"AvgDeposit"] = trial.loc[i,"Deposit Amt."]
        else:
            trial.loc[i,"AvgDeposit"] = 0
    
    trial['AvgWithDrawal'] = trial['AvgWithDrawal'].replace(np.nan, 0)
    trial['AvgDeposit'] = trial['AvgDeposit'].replace(np.nan, 0)
   
    trial["MonthYear"] = trial["Month"].str.cat(trial["Year"])
  
    grouped_months = trial["MonthYear"].unique()
    
    trial = trial.drop(["Month","Year"], axis=1)
      
    months = []
    monthly_transactions = []
    
    for i in grouped_months:
        months.append(i)
        temp_data = trial[trial["MonthYear"] == i].reset_index(drop=True)
        monthly_transactions.append(temp_data)
     
    monthly_transaction_history = dict(zip(months, monthly_transactions))

    monthly_transaction_history_details = pd.DataFrame(
        columns=['MONTH', 'NO. OF WITHDRAWALS', 'TOTAL WITHDRAWAL AMT.','AVG WITHDRAWAL AMT.', 'NO. OF DEPOSITS', 'TOTAL DEPOSIT AMT.','AVG DEPOSIT AMT.',
                 'OPENING BALANCE',
                 'CLOSING BALANCE', 'TOTAL LOANS', 'TOTAL WITHDRAWAL LOAN AMT.', 'BANK NAME'])

    for i in monthly_transaction_history.keys():
        monthly_transaction_history_details.loc[months.index(i), 'MONTH'] = i
        monthly_transaction_history_details.loc[months.index(i), 'NO. OF WITHDRAWALS'] = monthly_transaction_history[
                                                                                             i].loc[:,
                                                                                         'Withdrawal Amt.'].count()
        monthly_transaction_history_details.loc[months.index(i), 'NO. OF DEPOSITS'] = monthly_transaction_history[
                                                                                          i].loc[:,
                                                                                      'Deposit Amt.'].count()
        monthly_transaction_history_details.loc[months.index(i), 'OPENING BALANCE'] = \
            monthly_transaction_history[i].loc[0, 'Closing Balance']
        monthly_transaction_history_details.loc[months.index(i), 'CLOSING BALANCE'] = \
            monthly_transaction_history[i].loc[len(monthly_transaction_history[i]) - 1, 'Closing Balance']
        monthly_transaction_history_details.loc[months.index(i), 'TOTAL WITHDRAWAL AMT.'] = monthly_transaction_history[
                                                                                                i].loc[:,
                                                                                            'Withdrawal Amt.'].sum()
        monthly_transaction_history_details.loc[months.index(i), 'TOTAL DEPOSIT AMT.'] = monthly_transaction_history[
                                                                                             i].loc[:,
                                                                                         'Deposit Amt.'].sum()
        monthly_transaction_history_details.loc[months.index(i), 'AVG DEPOSIT AMT.'] = monthly_transaction_history[
                                                                                             i].loc[:,
                                                                                         'AvgDeposit'].mean()
        monthly_transaction_history_details.loc[months.index(i), 'AVG WITHDRAWAL AMT.'] = monthly_transaction_history[
                                                                                             i].loc[:,
                                                                                         'AvgWithDrawal'].mean()


    monthly_transaction_history_details_totalsum = monthly_transaction_history_details.sum()
    monthly_transaction_history_details_totalsum[0] = "TOTAL"
    monthly_transaction_history_details = monthly_transaction_history_details.append(monthly_transaction_history_details_totalsum, ignore_index=True)
    
    trial = trial.drop(["AvgDeposit","AvgWithDrawal"], axis=1)
    
    try:
        grouped_transctypes = trial.iloc[0:stop].groupby(['TransactionType'])
        loan_emi = grouped_transctypes.get_group('LOAN_EMI')
        loan_emi.reset_index(drop=True, inplace=True)

        loan_emi_transactions = []
        loan_emi_amounts = []
        for i in loan_emi.groupby('Withdrawal Amt.').groups.keys():
            loan_emi_amounts.append(i)
            loan_emi_transactions.append(pd.DataFrame(loan_emi.groupby('Withdrawal Amt.').get_group(i)))

        loan_history_per_amount = dict(zip(loan_emi_amounts, loan_emi_transactions))

        for z in loan_history_per_amount.keys():
            loan_history_per_amount[z]['Bank_Name'] = ''
            for x in list(bank_code_name.keys()):
                for y in range(len(loan_history_per_amount[z])):
                    if re.search(x, loan_history_per_amount[z].iloc[y, trans_val]):
                        loan_history_per_amount[z].iloc[y, -1] = bank_code_name[x]

        loan_history = pd.DataFrame()
        for i in loan_history_per_amount.keys():
            correct_transaction = pd.DataFrame(
                np.vstack([loan_history_per_amount[i].columns, loan_history_per_amount[i].iloc[:]]))
            loan_history = loan_history.append(correct_transaction)

        loan_history.columns = list(loan_history.iloc[0])
        loan_history.drop(loan_history.index[0], inplace=True)
        loan_history.reset_index(drop=True, inplace=True)
        loan_history_by_month = []
        loan_months = []
        for i in loan_history.groupby('MonthYear').groups.keys():
            loan_months.append(i)
            loan_history_by_month.append(pd.DataFrame(loan_history.groupby('MonthYear').get_group(i)))
            loan_history_monthly = dict(zip(loan_months, loan_history_by_month))

        for i in loan_history_monthly.keys():
            monthly_transaction_history_details.loc[months.index(i), 'TOTAL WITHDRAWAL LOAN AMT.'] = \
                loan_history_monthly[i].loc[:, 'Withdrawal Amt.'].sum()
            monthly_transaction_history_details.loc[months.index(i), 'TOTAL LOANS'] = loan_history_monthly[i].loc[:,
                                                                                      'Withdrawal Amt.'].count()
            monthly_transaction_history_details.loc[months.index(i), 'BANK NAME'] = list(
                set(loan_history_monthly[i].loc[:, 'Bank_Name']))
    except:
        loan_history = pd.DataFrame()
        loan_history_monthly = pd.DataFrame()

    trial['BANK NAME'] = ''
    for x in list(bank_code_name.keys()):
        for y in range(0, stop):
            if re.search(x, trial.iloc[y, trans_val]):
                trial.iloc[y, -1] = bank_code_name[x]

    rent_transactions = trial[trial.loc[:, 'Transaction Details'].str.contains('RENT', case=True, regex=True)]
    rent_transactions.reset_index(drop=True, inplace=True)

    credit_transactions = trial.sort_values('Deposit Amt.', ascending=False)
    credit_transactions.dropna(subset=['Deposit Amt.'], inplace=True)
    credit_transactions.reset_index(drop=True, inplace=True)

    recurring_transactions_debit = trial.iloc[0:stop].groupby(['Withdrawal Amt.'])
    recurring_transactions_amt_db = []
    recurring_transactions_db = []
    for i in recurring_transactions_debit.groups.keys():
        if i > 200 and recurring_transactions_debit.get_group(i).shape[0] > 1:
            recurring_transactions_amt_db.append(i)
            recurring_transactions_db.append(pd.DataFrame(recurring_transactions_debit.get_group(i)))

    debit_repeated = dict(zip(recurring_transactions_amt_db, recurring_transactions_db))

    recurring_transactions_credit = trial.iloc[0:stop].groupby(['Deposit Amt.'])
    recurring_transactions_amt_cr = []
    recurring_transactions_cr = []
    for i in recurring_transactions_credit.groups.keys():
        if i > 200 and recurring_transactions_credit.get_group(i).shape[0] > 1:
            recurring_transactions_amt_cr.append(i)
            recurring_transactions_cr.append(pd.DataFrame(recurring_transactions_credit.get_group(i)))

    credit_repeated = dict(zip(recurring_transactions_amt_cr, recurring_transactions_cr))

    credit_repeated_history = pd.DataFrame()
    for i in credit_repeated.keys():
        correct_transaction = pd.DataFrame(np.vstack([credit_repeated[i].columns, credit_repeated[i].iloc[:]]))
        credit_repeated_history = credit_repeated_history.append(correct_transaction)

    debit_repeated_history = pd.DataFrame()
    for i in debit_repeated.keys():
        correct_transaction = pd.DataFrame(np.vstack([debit_repeated[i].columns, debit_repeated[i].iloc[:]]))
        debit_repeated_history = debit_repeated_history.append(correct_transaction)

    return credit_transactions, monthly_transaction_history, loan_history_monthly, monthly_transaction_history_details, rent_transactions, \
           trial, loan_history, credit_repeated_history, debit_repeated_history
